### ターゲットとなるイオンの元素種を含むPOSCARのディレクトリパス一覧を取得

In [1]:
import numpy as np

In [2]:
target_npy_p = '../get_some_speceis_existed_poscar_path_list/C_O_existed_poscar_folder_path_list.npy'
C_O_existed_poscar_folder_path_list = np.load(target_npy_p, allow_pickle=True)
print(f"len(C_O_existed_poscar_folder_path_list): {len(C_O_existed_poscar_folder_path_list)}")

len(C_O_existed_poscar_folder_path_list): 205733


### ターゲットとなるイオンの元素種を含むPOSCAR.nnlistのディレクトリパス一覧を取得

In [45]:
add_str = '/nnlist_5/POSCAR.nnlist'
C_O_existed_poscar_nnlist_path_list = [str(p) + add_str for p in C_O_existed_poscar_folder_path_list]

### 炭酸イオンを含むか否かの判定アルゴリズムとファイル変換packageをダウンロード

In [55]:
from package_bond_search_algorithm.algolithm_bond_search_for_CO3 import concat_filter
from package_bond_search_algorithm.package_file_conversion.nnlist2df import nnlist2df

#### サンプルの1ファイルでPOSCAR.nnlistをDataFrameに変換できるかテスト

In [85]:
nnlist_p = C_O_existed_poscar_nnlist_path_list[8]
print(nnlist_p)
df_nnlist = nnlist2df(nnlist_p)
df_nnlist.head(1)

/mnt/ssd_elecom_c2c_960gb/cif/1/00/00/1000033/nnlist_5/POSCAR.nnlist


,central_atom_id,neighboring_atom_id,rel_distance,rel_x,rel_y,rel_z,unitcell_x,unitcell_y,unitcell_z,central_atom_symbol,neighboring_atom_symbol
0,1,1,0.000000,0.000000,0.000000,0.000000,0,0,0,Ba,Ba


#### テストとしてalgorithmを用いてPOSCAR.nnlistがCO3を含むか判定

In [76]:
concat_filter(df_nnlist=df_nnlist)

True

### POSCAR.nnlistをDataFrameに変換する処理と，変換したDataFrameをalgolithmを用いて判定する処理を関数にまとめる

In [84]:
def wrap_nnlist2df_and_concat_filter(nnlist_p):
    """
    This func() executes two func()s.: nnlistdf() and concat_filter().
    
    Usage:
    ------
    wrap_nnlist2df_and_concat_filter(nnlist_p=nnlist_p)
    
    Parameter:
    -----------
    nnlist_p: str or PosixPath
    
    Return:
    -------
    bool: True or False
    """
    df_nnlist = nnlist2df(nnlist_p)
    bool_ = concat_filter(df_nnlist=df_nnlist)

    return bool_

In [86]:
wrap_nnlist2df_and_concat_filter(nnlist_p=nnlist_p)

True

### 処理を並列化

In [92]:
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
pp = Pool(cpu_count() - 1)
total = len(C_O_existed_poscar_nnlist_path_list)
try:
    bool_ion_contain = list(tqdm(pp.imap(wrap_nnlist2df_and_concat_filter, C_O_existed_poscar_nnlist_path_list[0:50]), total=total))
finally:
    pp.close()
    pp.join()

  0%|                                                                                                                   | 50/205733 [00:00<29:17, 117.05it/s]


In [106]:
ion_contained_poscar_nnlist_path_list = np.array(C_O_existed_poscar_nnlist_path_list[0:50])[bool_ion_contain]
from pathlib import Path
ion_contained_poscar_folder_path_list = [Path(os.path.split(Path(os.path.split(Path(p))[0]))[0]) for p in ion_contained_poscar_nnlist_path_list]

### ion_contained_poscar_folder_path_listを保存

In [108]:
saved_npy_fname = 'CO3_contained_poscar_folder_path_list.npy'
np.save(saved_npy_fname, ion_contained_poscar_folder_path_list)

### 保存した.npyファイルをload

In [109]:
np.load(saved_npy_fname, allow_pickle=True)

array([PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/00/1000033'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/00/1000052'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000320'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000330'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000331'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000361'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000362'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000372'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/03/1000394'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/04/1000450'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/04/1000459'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/04/1000471'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/04/1000477'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/00/70/1007035'),
       PosixPath('/mnt/ssd_elecom_c2c_960gb/cif/1/01/02/101021